In [1]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "gurobi","gcg"],  # modules to install
    license_uuid="635a7a6b-17a0-4d5a-991e-74343f82957d",  # license to use
)  # instantiate AMPL object and register magics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.1 MB/s eta 0:00:00
Licensed to Bundle #6920.7400 expiring 20250630: Decision and Learning Under Uncertainty, Prof. Viet Hung Nguyen, Clermont Auvergne University.


In [2]:
%%ampl_eval
reset;
set ITEMS;

param weight{ITEMS};
param value{ITEMS};
param max_weight;

var x{ITEMS} binary;

maximize total_value: sum{i in ITEMS} value[i] * x[i];

subject to weight_constraint: sum{i in ITEMS} weight[i] * x[i] <= max_weight;

In [3]:
from random import randint

items = list(range(1, 10 + 1))
ampl.set["ITEMS"] = items
ampl.param["weight"] = {i: randint(10, 20) for i in items}
ampl.param["value"] = {i: randint(100, 200) for i in items}
ampl.param["max_weight"] = 100
ampl.option["solver"] = "highs"
ampl.solve()
ampl.get_data("x").to_pandas()

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 1098
3 simplex iterations
1 branching nodes


,x
1,1
2,0
3,1
4,1
5,0
6,0
7,1
8,1
9,1
10,1


In [4]:
%%ampl_eval
reset;
param n:=10000;
set S :=1 .. n;
param p{s in S} default 1/card(S);
param w1{S} := Uniform(1,4);
param w2{S} :=Uniform(1/3,1);

param PENALTY := 5;
var x1 >= 0;
var x2 >= 0;
var y1{S} >= 0;
var y2{S} >= 0;

minimize obj: x1 + x2 + sum{s in S} p[s] * PENALTY * (y1[s] + y2[s]);
subject to c1{s in S}:
w1[s] * x1 + x2 + y1[s] >= 7;
subject to c2{s in S}:
w2[s] * x1 + x2 + y2[s] >= 4;
option solver highs;
solve;
display x1, x2;

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 6.222845464
3562 simplex iterations
0 barrier iterations
x1 = 2.76989
x2 = 2.97396



In [5]:
%%ampl_eval
reset;
var x1 >= 0;
var x2 >= 0;

minimize ObjPlusRecourse:
x1 + x2;
subject to c1:
x1 + x2 >= 7;
subject to c2:
(1/3)*x1 + x2 >= 4;
solve;
display x1, x2;


HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 7
1 simplex iterations
0 barrier iterations
x1 = 0
x2 = 7



In [6]:
%%writefile farmer.mod
set Crop;
set Scenario;
set Fields;

param Yield{Crop};
param PlantingCost{Crop};
param SellingPrice{Crop};
param MinReq{Crop};
param Quota{Crop} default Infinity;
param SellingPriceAfterQuota{Crop} default 0;

param FieldAcres{Fields};

param Multiplier{Scenario};

param PurchasePrice{i in Crop} := 1.4*SellingPrice[i];

var FieldPlanted{Crop,Fields} binary;
var Bought{Crop,Scenario}>=0;
var Sold{Crop, Scenario} >=0;
var SoldAboveQuota{Crop,Scenario} >=0;

minimize TotalCost:
	(1/3)* (sum{j in Scenario}(sum{i in Crop}(sum{k in Fields}(FieldPlanted[i,k]*FieldAcres[k]*PlantingCost[i])
	+ Bought[ i,j ] * PurchasePrice[ i ]
	-( Sold[ i,j ] - SoldAboveQuota[ i,j ] ) * SellingPrice[ i ]
	- SoldAboveQuota[ i,j ] * SellingPriceAfterQuota[ i ]))) ;



subject to FieldCapacity{k in Fields}:
	sum{i in Crop}FieldPlanted[i,k] = 1;


subject to Quotas{i in Crop, j in Scenario}:
	SoldAboveQuota[i,j]>=Sold[i,j] - Quota[i];

subject to MinimumRequirements{i in Crop, j in Scenario}:
	sum{k in Fields}FieldPlanted[i,k]*FieldAcres[k]*Yield[i]*Multiplier[j]
	+ Bought[i,j] - Sold[i,j] - SoldAboveQuota[i,j] >= MinReq[i];

Writing farmer.mod


In [7]:
%%writefile farmer.dat
set Crop = wheat corn sugarbeets;
set Scenario = low average high;
set Fields = 1 2 3 4;

param:		Yield	PlantingCost	SellingPrice		  MinReq	Quota	SellingPriceAfterQuota :=
	wheat		2.5		150				170					200				.				.
	corn		3		230				150					240				.				.
	sugarbeets	20		260				36					0				6000			10;

param FieldAcres =
1 185
2 145
3 105
4 65;

param Multiplier =
	low			.8
	average		1
	high		1.2;

Writing farmer.dat


In [8]:
%%writefile farmer.run
option solver highs;
model farmer.mod;
data farmer.dat;
solve;
display FieldPlanted;
display Bought;
display Sold;
display SoldAboveQuota;

Writing farmer.run


In [9]:
ampl = AMPL()  # create an AMPL object
ampl.read("farmer.run")  # Execute the .run file

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective -107975
44 simplex iterations
1 branching nodes
FieldPlanted :=
corn       1   0
corn       2   0
corn       3   1
corn       4   0
sugarbeets 1   1
sugarbeets 2   0
sugarbeets 3   0
sugarbeets 4   1
wheat      1   0
wheat      2   1
wheat      3   0
wheat      4   0
;

Bought :=
corn       average   0
corn       high      0
corn       low       0
sugarbeets average   0
sugarbeets high      0
sugarbeets low       0
wheat      average   0
wheat      high      0
wheat      low       0
;

Sold :=
corn       average     75
corn       high       138
corn       low         12
sugarbeets average   5000
sugarbeets high      6000
sugarbeets low       4000
wheat      average    162.5
wheat      high       235
wheat      low         90
;

SoldAboveQuota :=
corn       average   0
corn       high      0
corn       low       0
sugarbeets average   0
sugarbeets high      0
sugarbeets low       0
wheat      average   0
wheat      high 

In [41]:
%%writefile Lshape.mod
param nSCEN;
set SCEN;

param nCut >= 0 integer;
param scenIx integer;

var x1 >= 0, := 2;
var x2 >= 0, := 2;
var theta >= 0;  #(Since q >=0, y >= 0)

var y1{SCEN} >= 0;
var y2{SCEN} >= 0;


param q1;
param q2;
param p{SCEN} default 1/card(SCEN);
param omega1{SCEN};
param omega2{SCEN};

param tcoef0{SCEN} default 0;
param tcoef1{SCEN} default 0;
param tcoef2{SCEN} default 0;

param coef0{1..nCut} default 0;
param coef1{1..nCut} default 0;
param coef2{1..nCut} default 0;


minimize SubObj{s in SCEN}:
         q1 * y1[s] + q2 * y2[s];

subject to c1{s in SCEN}:
        omega1[s] * x1 + x2 + y1[s] >= 7;

subject to c2{s in SCEN}:
        omega2[s] * x1 + x2 + y2[s] >= 4;


minimize MasterObj:
	 x1 + x2 + theta;

subject to Cuts {k in 1..nCut}:
	theta >= coef0[k] - coef1[k] * x1 - coef2[k] * x2;


Overwriting Lshape.mod


In [11]:
%%writefile Lshape.dat
param q1 := 2;
param q2 := 2;

param nSCEN := 3;
set SCEN := 1 2 3;

param omega1 :=
1 1
2 2.5
3 4
;

param omega2 :=
1 0.33333333
2 0.66666667
3 1
;

param p :=
1 0.33333333
2 0.33333333
3 0.33333333
;

Writing Lshape.dat


In [42]:
%%writefile Lshape.run
option solver highs;
model Lshape.mod;
data Lshape.dat;

problem Master: x1, x2, theta, MasterObj, Cuts;

problem Sub {s in SCEN}:
	y1[s], y2[s], SubObj[s], c1[s], c2[s];

param g;
let nCut := 0;
repeat {
    printf "\nITERATION: %d\n\n", nCut;
    let nCut := nCut + 1;
    let scenIx := 0;
    let g:=0;
    for {s in SCEN} {
	solve Sub[s];
	let scenIx := scenIx + 1;
	if SubObj[s]>0.001 then {
	let tcoef0[s] := c1[s].dual * 7 + c2[s].dual * 4;
	let tcoef1[s] := c1[s].dual * omega1[s] + c2[s].dual * omega2[s];
	let tcoef2[s] := c1[s].dual + c2[s].dual;
	}
	else{
	let tcoef0[s] := 0;
	let tcoef1[s] := 0;
	let tcoef2[s] := 0;
	}
	let g:=g+p[s]*(tcoef0[s]-tcoef1[s]*x1-tcoef2[s]*x2);
	printf "SCENARIO %d\n", scenIx;
	display SubObj[s], y1[s], y2[s], omega1[s], omega2[s], x1, x2;
	display tcoef0[s], tcoef1[s], tcoef2[s];

    }
    if theta >g+0.000001 then{
		printf("Problem solved to optimality\n");
		break;
	}
    let coef0[nCut] := sum {s in SCEN} p[s] * tcoef0[s];
    let coef1[nCut] := sum {s in SCEN} p[s] * tcoef1[s];
    let coef2[nCut] := sum {s in SCEN} p[s] * tcoef2[s];

    display coef0[nCut], coef1[nCut], coef2[nCut];
    solve Master;
    printf "\n";
    display MasterObj, theta, x1, x2;
    if nCut>=20 then{
		printf("Problem not solved to optimality\n");
		break;
	}
};

Overwriting Lshape.run


In [43]:
ampl = AMPL()  # create an AMPL object
ampl.read("Lshape.run")  # Execute the .run file


ITERATION: 0

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 8.66666668
0 simplex iterations
0 barrier iterations
SCENARIO 1
SubObj[s] = 8.66667
y1[s] = 3
y2[s] = 1.33333
omega1[s] = 1
omega2[s] = 0.333333
x1 = 2
x2 = 2

tcoef0[s] = 22
tcoef1[s] = 2.66667
tcoef2[s] = 4

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 1.33333332
0 simplex iterations
0 barrier iterations
SCENARIO 2
SubObj[s] = 1.33333
y1[s] = 0
y2[s] = 0.666667
omega1[s] = 2.5
omega2[s] = 0.666667
x1 = 2
x2 = 2

tcoef0[s] = 8
tcoef1[s] = 1.33333
tcoef2[s] = 2

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 0
0 simplex iterations
0 barrier iterations
SCENARIO 3
SubObj[s] = 0
y1[s] = 0
y2[s] = 0
omega1[s] = 4
omega2[s] = 1
x1 = 2
x2 = 2

tcoef0[s] = 0
tcoef1[s] = 0
tcoef2[s] = 0

coef0[nCut] = 10
coef1[nCut] = 1.33333
coef2[nCut] = 2

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 5
0 simplex iterations
0 barrier iterations

MasterObj = 5
theta

In [25]:
%%writefile benders_multi.mod
param nSCEN;
set SCEN;

param nCut >= 0 integer;
param scenIx integer;

var x1 >= 0, := 2;
var x2 >= 0, := 2;
var theta{SCEN} >= 0;  #(Since q >=0, y >= 0)

var y1{SCEN} >= 0;
var y2{SCEN} >= 0;


param q1;
param q2;
param p{SCEN} default 1/card(SCEN);
param omega1{SCEN};
param omega2{SCEN};

param tcoef0{SCEN} default 0;
param tcoef1{SCEN} default 0;
param tcoef2{SCEN} default 0;

param sce{1..nCut} default 0;
param coef0{1..nCut} default 0;
param coef1{1..nCut} default 0;
param coef2{1..nCut} default 0;


minimize SubObj{s in SCEN}:
         q1 * y1[s] + q2 * y2[s];

subject to c1{s in SCEN}:
        omega1[s] * x1 + x2 + y1[s] >= 7;

subject to c2{s in SCEN}:
        omega2[s] * x1 + x2 + y2[s] >= 4;


minimize MasterObj:
	 x1 + x2 + sum{s in SCEN}p[s]*theta[s];

subject to Cuts {k in 1..nCut}:
	theta[sce[k]] >= coef0[k] - coef1[k] * x1 - coef2[k] * x2;


Writing benders_multi.mod


In [26]:
%%writefile benders_multi.run
option solver highs;
model benders_multi.mod;
data Lshape.dat;

problem Master: x1, x2, theta, MasterObj, Cuts;

problem Sub {s in SCEN}:
	y1[s], y2[s], SubObj[s], c1[s], c2[s];

param g;
param iter;
param opt;
let nCut := 0;
let iter:=0;

repeat {
    printf "\nITERATION: %d\n\n", iter;
    let iter:=iter+1;
    let scenIx := 0;
    let g:=0;
    let opt:=0;
    for {s in SCEN} {
		solve Sub[s];
		let scenIx := scenIx + 1;
		printf "SCENARIO %d\n", scenIx;
		display SubObj[s], y1[s], y2[s], omega1[s], omega2[s], x1, x2;
		let tcoef0[s] := c1[s].dual * 7 + c2[s].dual * 4;
		let tcoef1[s] := c1[s].dual * omega1[s] + c2[s].dual * omega2[s];
		let tcoef2[s] := c1[s].dual + c2[s].dual;
		let g:=tcoef0[s]-tcoef1[s]*x1-tcoef2[s]*x2;
		if theta[s]<g-0.00001 then{
			let nCut := nCut + 1;
			let sce[nCut] :=s;
			let coef0[nCut] := tcoef0[s];
			let coef1[nCut] := tcoef1[s];
			let coef2[nCut] := tcoef2[s];
			display coef0[nCut], coef1[nCut], coef2[nCut];
		}
		else{
			let opt:=opt+1;
		}
    }
    if opt=card(SCEN) then{
		printf("Problem solved to optimality\n");
		break;
	}

    solve Master;
    printf "\n";
    display MasterObj, theta, x1, x2;
    if nCut>=60 then{
		printf("Problem not solved to optimality\n");
		break;
	}
};

Writing benders_multi.run


[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)

In [27]:
ampl = AMPL()  # create an AMPL object
ampl.read("benders_multi.run")  # Execute the .run file


ITERATION: 0

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 8.66666668
0 simplex iterations
0 barrier iterations
SCENARIO 1
SubObj[s] = 8.66667
y1[s] = 3
y2[s] = 1.33333
omega1[s] = 1
omega2[s] = 0.333333
x1 = 2
x2 = 2

coef0[nCut] = 22
coef1[nCut] = 2.66667
coef2[nCut] = 4

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 1.33333332
0 simplex iterations
0 barrier iterations
SCENARIO 2
SubObj[s] = 1.33333
y1[s] = 0
y2[s] = 0.666667
omega1[s] = 2.5
omega2[s] = 0.666667
x1 = 2
x2 = 2

coef0[nCut] = 8
coef1[nCut] = 1.33333
coef2[nCut] = 2

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 0
0 simplex iterations
0 barrier iterations
SCENARIO 3
SubObj[s] = 0
y1[s] = 0
y2[s] = 0
omega1[s] = 4
omega2[s] = 1
x1 = 2
x2 = 2

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 5.5
1 simplex iterations
0 barrier iterations

MasterObj = 5.5

theta [*] :=
1  0
2  0
3  0
;

x1 = 0
x2 = 5.5


ITERATION: 1

HiGHS 1.8.1: 